In [21]:
# Read data from file
import numpy as np
def read(filename):
    with open(filename, 'r') as f:
        N, K = map(int, f.readline().split())
        d = [0] + list(map(float, f.readline().split()))
        t = np.zeros((N + 1, N + 1))
        for i in range(N + 1):
            t[i] = list(map(float, f.readline().split()))

    return N, K, d, t

def creat_cost(N, d, t):
    cost_matrix = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(N + 1):
            if i != j:
                cost_matrix[i][j] = d[j] + t[i][j]

    return cost_matrix

def print_result(K, N, x, objective, time_pro, filename):
    with open(filename, 'w') as f:
        f.write("Objective: " + str(objective) + '\n')
        f.write("Time processing: " + str(time_pro) + '\n')
        f.write("Solution: \n")
        f.write(str(K) + '\n')
        for k in range(K):
            f.write(str(int(sum(x[i][j][k] for j in range(N + 1) for i in range(N + 1)))) + '\n')
            current_pos = 0
            f.write(str(0) + ' ')

            while True:
                for j in range(N + 1):
                    if x[current_pos][j][k] == 1:
                        f.write(str(j) + ' ')
                        current_pos = j
                        break
                if current_pos == 0:
                    break

            f.write('\n')

def print_no_solution(filename):
    with open(filename, 'w') as f:
        f.write("No solution")

        

In [22]:
from ortools.linear_solver import pywraplp
def solve_maintenance_problem(file_data):
    # Read data from file
    data_path = "Data\\" + file_data
    N, K, d, t = read(data_path)
    cost_matrix = creat_cost(N, d, t)


    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Biến quyết định x[i][j][k]: Nhân viên k di chuyển từ i đến j hay không
    x = [[[solver.IntVar(0, 1, f'x_{i}_{j}_{k}') for k in range(K )] for j in range(N + 1)] for i in range(N + 1)]

    # Biến p[i][k]: Thứ tự của khách hàng i trong lịch trình của nhân viên k
    p = [[solver.IntVar(0, N, f'p_{i}_{k}') for k in range(K)] for i in range(N + 1)]

    # Tạo biến y[k] để chỉ trạng thái hoạt động của nhân viên k
    y = [solver.IntVar(0, 1, f'y_{k}') for k in range(K)]

    # Ràng buộc: Mỗi khách hàng chỉ được bảo trì bởi một nhân viên
    for j in range(1, N + 1):
        solver.Add(sum(x[i][j][k] for i in range(N + 1) for k in range(K)) == 1)

    # Ràng buộc: Mỗi nhân viên chỉ di chuyển nhiều nhất là 1 chuyến
    for k in range(K):
        solver.Add(sum(x[0][j][k] for j in range(1, N + 1)) <= y[k])


    # Ràng buộc: Đến và rời khỏi khách hàng bởi cùng một nhân viên
    for j in range(0, N + 1):
        for k in range(K):
            solver.Add(sum(x[i][j][k] for i in range(N + 1)) == sum(x[j][i][k] for i in range(N + 1)))


    # Ràng buộc:
    for k in range(K):
        for i in range(0, N + 1):
            solver.Add(x[i][i][k] == 0)

    # Ràng buộc: Mỗi nhân viên bắt đầu từ 0 và kết thúc tại 0
    for k in range(K):
        solver.Add(sum(x[0][j][k] for j in range(1, N + 1)) == y[k])
        solver.Add(sum(x[i][0][k] for i in range(1, N + 1)) == y[k])

    # Ràng buộc: Nếu x[i][j][k] = 1 thì p[i][k] < p[j][k]
    for k in range(K):
        for i in range(0,N + 1):
            for j in range(1, N + 1):
                if i != j:
                    solver.Add(1000*(x[i][j][k] - 1) + p[i][k] + 1 <= p[j][k])


    # Mục tiêu: Tối thiểu hóa thời gian làm việc nhiều nhất của một nhân viên
    max_work_time = solver.NumVar(0, solver.infinity(), 'max_work_time')
    for k in range(K):
        solver.Add(max_work_time >= (sum(x[i][j][k] * cost_matrix[i][j] for i in range(N + 1) for j in range(N + 1))))

    solver.Minimize(max_work_time)

    # Giải bài toán
    status = solver.Solve()

    filename_result = "Result_MIP\\" + file_data

    # In kết quả
    if status == pywraplp.Solver.OPTIMAL:
        # In ra hanh trinh cua tung xe
        objective_value = solver.Objective().Value()
        x = [[[x[i][j][k].solution_value() for k in range(K)] for j in range(N + 1)] for i in range(N + 1)]
        time_pro = solver.WallTime()
        print_result(K, N, x, objective_value, time_pro, filename_result)
    else:
        print_no_solution(filename_result)


In [23]:
solve_maintenance_problem("data_5_2.txt")